

a)

In [2]:
import requests
import pandas as pd
import time
from datetime import datetime


top_25_coins = [
    'BTC', 'ETH', 'USDT', 'BNB', 'SOL', 'USDC', 'XRP', 'TON', 'DOGE', 'ADA',
    'AVAX', 'WETH', 'SHIB', 'DOT', 'BCH', 'TRX', 'LINK', 'MATIC', 'ICP', 'WBTC',
    'LTC', 'NEAR', 'DAI', 'UNI', 'LEO'
]
API_KEY = 'my_api'  
limit_days = 1095  # ~3 years

def get_daily_price_data(symbol: str, currency='USD'):
    url = f'https://min-api.cryptocompare.com/data/v2/histoday'
    params = {
        'fsym': symbol,
        'tsym': currency,
        'limit': limit_days,
        'api_key': API_KEY
    }

    try:
        response = requests.get(url, params=params)
        data = response.json()
        if data['Response'] == 'Success':
            df = pd.DataFrame(data['Data']['Data'])
            df['time'] = pd.to_datetime(df['time'], unit='s')
            df['symbol'] = symbol
            return df
        else:
            print(f"Failed to get data for {symbol}: {data.get('Message', '')}")
            return None
    except Exception as e:
        print(f"Error fetching {symbol}: {e}")
        return None

# Collect data for top 25 coins
all_data = []

for coin in top_25_coins:
    print(f"Fetching {coin}...")
    df = get_daily_price_data(coin)
    if df is not None:
        all_data.append(df)
    time.sleep(1)  # Be gentle with the API

# Combine into one DataFrame
crypto_data = pd.concat(all_data, ignore_index=True)

# Save to CSV
crypto_data.to_csv("top25_crypto_3years.csv", index=False)
print("Data collection complete. Saved to 'top25_crypto_3years.csv'.")


Fetching BTC...
Fetching ETH...
Fetching USDT...
Fetching BNB...
Fetching SOL...
Fetching USDC...
Fetching XRP...
Fetching TON...
Fetching DOGE...
Fetching ADA...
Fetching AVAX...
Fetching WETH...
Fetching SHIB...
Fetching DOT...
Fetching BCH...
Fetching TRX...
Fetching LINK...
Fetching MATIC...
Fetching ICP...
Fetching WBTC...
Fetching LTC...
Fetching NEAR...
Fetching DAI...
Fetching UNI...
Fetching LEO...
Data collection complete. Saved to 'top25_crypto_3years.csv'.



b)

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_1samp

# Load price data 
df = pd.read_csv("top25_crypto_3years.csv")
df['time'] = pd.to_datetime(df['time'])

# Strategy Parameters
sma_short = 45
sma_long = 90
trading_days = 252  # For Sharpe ratio

# Results storage
results = []

# Loop through each crypto
for symbol in df['symbol'].unique():
    data = df[df['symbol'] == symbol].sort_values('time').copy()
    
    # Compute daily returns
    data['return'] = data['close'].pct_change()

    # Compute moving averages
    data['SMA1'] = data['close'].rolling(window=sma_short).mean()
    data['SMA2'] = data['close'].rolling(window=sma_long).mean()

    # Generate position: 1 if SMA1 > SMA2, else 0
    data['position'] = (data['SMA1'] > data['SMA2']).astype(int)
    data['strategy_return'] = data['position'].shift(1) * data['return']
    
    # Drop missing values
    data = data.dropna()

    # Compute Sharpe Ratio
    avg_ret = data['strategy_return'].mean()
    std_ret = data['strategy_return'].std()
    sharpe = (avg_ret / std_ret) * np.sqrt(trading_days) if std_ret != 0 else 0

    # T-test: is mean return significantly different from 0
    t_stat, p_value = ttest_1samp(data['strategy_return'].dropna(), 0)

    # Store result
    results.append({
        'Crypto': symbol,
        'Sharpe Ratio': sharpe,
        'p-value': p_value
    })

# Convert to DataFrame
results_df = pd.DataFrame(results)
results_df.sort_values(by='Sharpe Ratio', ascending=False, inplace=True)

# Save or display
from IPython.display import display
display(results_df)


/Users/anaconda3/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/anaconda3/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/anaconda3/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / 

,Crypto,Sharpe Ratio,p-value
7,TON,1.135137,0.051342
23,UNI,1.067310,0.729633
22,DAI,0.769175,0.428355
3,BNB,0.724083,0.203079
19,WBTC,0.718804,0.155811
11,WETH,0.609623,0.342079
14,BCH,0.000000,NaN
2,USDT,-0.395777,0.449961
5,USDC,-0.525714,0.322621
24,LEO,-2.770904,0.372741




The table shows the results from the strategy, Simple Moving Average crossover. The metrics i got was sharpe ratio and the P-value. some of the cryptocurrencies showed promising risk adjusting return when using the Simple Moving Average crossover method, while others gave alot of inconsistens results. This highligts the importance of controlling the strategys robutness, not trusting backtest-results too much and ensure data quality.










c)

In [ ]:
import pandas as pd
import numpy as np
import random
from statsmodels.api import OLS, add_constant
from scipy.stats import ttest_1samp
from datetime import datetime
from IPython.display import display

# Load data
df = pd.read_csv("top25_crypto_3years.csv")
df['time'] = pd.to_datetime(df['time'])

# Filter only coins with enough data
valid_coins = df.groupby("symbol").filter(lambda x: len(x) > 400).symbol.unique()

# Randomly pick 3 non-overlapping pairs
random.seed(42)
coin_list = list(valid_coins)
random.shuffle(coin_list)
pairs = [(coin_list[i], coin_list[i+1]) for i in range(0, 6, 2)]

results = []

# Strategy parameters
lookback = 20
band_multiplier = 2
trading_days = 252

# Loop through all 3 pairs
for coin1, coin2 in pairs:
    pair_df = df[df['symbol'].isin([coin1, coin2])]
    pivot_df = pair_df.pivot(index='time', columns='symbol', values='close').dropna()

    # Make sure both coins exist in data
    if coin1 not in pivot_df.columns or coin2 not in pivot_df.columns:
        continue

    # Regression to get beta
    X = add_constant(pivot_df[coin2])
    model = OLS(pivot_df[coin1], X).fit()
    beta = model.params[coin2]

    # Spread & Bollinger Bands
    pivot_df['spread'] = pivot_df[coin1] - beta * pivot_df[coin2]
    pivot_df['SMA'] = pivot_df['spread'].rolling(window=lookback).mean()
    pivot_df['STD'] = pivot_df['spread'].rolling(window=lookback).std()
    pivot_df['upper'] = pivot_df['SMA'] + band_multiplier * pivot_df['STD']
    pivot_df['lower'] = pivot_df['SMA'] - band_multiplier * pivot_df['STD']

    # Trading logic
    position = []
    in_pos = 0
    for i in range(len(pivot_df)):
        row = pivot_df.iloc[i]
        if in_pos == 0:
            if row['spread'] < row['lower']:
                in_pos = 1
            elif row['spread'] > row['upper']:
                in_pos = -1
        elif in_pos == 1:
            if row['spread'] >= row['SMA']:
                in_pos = 0
        elif in_pos == -1:
            if row['spread'] <= row['SMA']:
                in_pos = 0
        position.append(in_pos)

    pivot_df['position'] = position
    pivot_df['ret1'] = pivot_df[coin1].pct_change()
    pivot_df['ret2'] = pivot_df[coin2].pct_change()
    pivot_df['strategy_ret'] = pivot_df['position'].shift(1) * (pivot_df['ret1'] - beta * pivot_df['ret2'])

    pivot_df.dropna(inplace=True)

    if len(pivot_df) < 30:
        continue

    # Sharpe + t-test
    mean_ret = pivot_df['strategy_ret'].mean()
    std_ret = pivot_df['strategy_ret'].std()
    sharpe_ratio = (mean_ret / std_ret) * np.sqrt(trading_days) if std_ret != 0 else 0
    t_stat, p_value = ttest_1samp(pivot_df['strategy_ret'], 0)

    results.append({
        'Crypto Pair': f"{coin1}/{coin2}",
        'Sharpe Ratio': sharpe_ratio,
        'p-value': p_value
    })

# Results as a DataFrame
results_df = pd.DataFrame(results)
display(results_df)


The above results summarize the performance of a Bollinger Bands-based pairs trading strategy applied to three randomly selected cryptocurrency pairs. The strategy uses the spread between two coins, adjusted by a hedge ratio (β), and trades on deviations from the 20-day moving average. The Sharpe ratio measures the risk-adjusted return of the strategy, and the p-value tests whether the average return is significantly different from zero. Statistically significant Sharpe ratios indicate potential profitability, though practical implementation should account for transaction costs and market frictions.

From the table i can conclude that the strategy was not robust across the 3 pairs.

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.api import OLS, add_constant
from scipy.stats import ttest_1samp
from itertools import product
from IPython.display import display

# Load your data
df = pd.read_csv("top25_crypto_3years.csv")
df['time'] = pd.to_datetime(df['time'])

# Choose crypto pair
coin1 = 'BTC'
coin2 = 'ETH'

# Prepare price data
pair_df = df[df['symbol'].isin([coin1, coin2])]
pivot_df = pair_df.pivot(index='time', columns='symbol', values='close').dropna()

# Calculate hedge ratio β
X = add_constant(pivot_df[coin2])
model = OLS(pivot_df[coin1], X).fit()
beta = model.params[coin2]

# Grid search over (n, m)
results = []
param_grid = list(product([10, 20, 30], [2, 4, 6]))

for n, m in param_grid:
    df_copy = pivot_df.copy()
    df_copy['spread'] = df_copy[coin1] - beta * df_copy[coin2]
    df_copy['SMA'] = df_copy['spread'].rolling(window=n).mean()
    df_copy['STD'] = df_copy['spread'].rolling(window=n).std()
    df_copy['upper'] = df_copy['SMA'] + m * df_copy['STD']
    df_copy['lower'] = df_copy['SMA'] - m * df_copy['STD']

 
    pos = []
    state = 0
    for i in range(len(df_copy)):
        row = df_copy.iloc[i]
        if state == 0:
            if row['spread'] < row['lower']:
                state = 1
            elif row['spread'] > row['upper']:
                state = -1
        elif state == 1 and row['spread'] >= row['SMA']:
            state = 0
        elif state == -1 and row['spread'] <= row['SMA']:
            state = 0
        pos.append(state)

    df_copy['position'] = pos
    df_copy['ret1'] = df_copy[coin1].pct_change()
    df_copy['ret2'] = df_copy[coin2].pct_change()
    df_copy['strategy_ret'] = df_copy['position'].shift(1) * (df_copy['ret1'] - beta * df_copy['ret2'])
    df_copy.dropna(inplace=True)

    if len(df_copy) < 30:
        continue

    # Metrics
    mean_ret = df_copy['strategy_ret'].mean()
    std_ret = df_copy['strategy_ret'].std()
    sharpe = (mean_ret / std_ret) * np.sqrt(252) if std_ret != 0 else 0
    t_stat, p_val = ttest_1samp(df_copy['strategy_ret'], 0)

    results.append({
        'n': n, 'm': m, 'Sharpe Ratio': sharpe, 'p-value': p_val
    })

grid_df = pd.DataFrame(results)
grid_df.sort_values('Sharpe Ratio', ascending=False, inplace=True)

display(grid_df)


While the Bollinger Bands pairs trading strategy produced promising Sharpe ratios in the backtest, its performance in a live trading setting is likely to be lower due to several practical limitations.
The results from the grid search confirm that the Bollinger Bands strategy is highly sensitive to parameter selection. Only a subset of combinations delivered meaningful Sharpe ratios, with most failing to generate trades or producing insignificant results. Many other parameter combinations yielded no returns or trades, highlighting the importance of proper tuning and the risk of overfitting. These findings are consistent with academic research, such as Danilo & Martin (2024), who also found that only a subset of configurations and pairs produced meaningful returns.



e) 
Based on the results obtained in part (c), the Bollinger Bands-based pairs trading strategy shows limited viability when talking about a live trading enviroment. While only some parameter combinations was producing positive sharpe ratios, there were not achived any statistically significant results. The explanations could be, that the returns may have been by chance where I would assume it wouldn’t persist out of sample. 
When trading live there could be other factors in play that could reduce the effectiveness even more. The grid search in part (d) increase the risk of overfitting, when the parameters were optimized. This can be backed up with Danilo and Martin (2024). Their optimized Bollinger Bands strategy achieved a strong median Sharpe ratio of 1.70 across 229 crypto pairs, only a subset of combinations yielded meaningful trades, and many produced no actionable signals. This reflects a key risk: even well-tuned strategies may not generalize reliably across pairs or time periods.
Overall, while the strategy may have academic merit and theoretical appeal, its real-world viability appears limited based on the results in part (c) and the insights provided by Danilo and Martin (2024). To improve robustness, the strategy would need to be validated through forward testing, out-of-sample performance, and strong risk management controls.







Litteratur used: 
Danilo, L., & Martin, F. (2024, November). Optimized pairs-trading strategies in the cryptocurrencies market using genetic algorithms and cointegration. ResearchGate.